In [17]:
# icecube_sim_to_data.ipynb
# Authors: Stephan Meighen-Berger
# Converts the new fluxes to "real" data

In [18]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import csv
from iminuit import Minuit
from scipy.interpolate import UnivariateSpline

In [19]:
# picture path
PICS = '../pics/'

In [20]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [21]:
# Plotting standards
std_size = 6.
fontsize = 20.
lw=1.
h_length=1.
mark_s = 10
# params
mag_fit = 1.
colors = ['#fef0d9', '#fdcc8a', '#fc8d59', '#e34a33', '#b30000']
alphas = [0.2, 0.4, 0.6, 0.8, 1.]
# labels_mass = ['1\;TeV', '100\;TeV', '10\;PeV', '30\;TeV']
linest = ['-', '--', '-.', ':']

In [22]:
# Constants
minutes = 60.
days = 60. * 24

In [23]:
def ice_parser(filename):
    store = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row_num, row in enumerate(reader):
            if row_num == 0:
                continue
            store.append(row[0].split())
    store = np.array(store, dtype=float)
    return store
# log10(E_nu/GeV)_min, log10(E_nu/GeV)_max, Dec_nu_min[deg], Dec_nu_max[deg], A_Eff[cm^2]
eff_areas = [
    '../data/icecube_10year_ps/irfs/IC40_effectiveArea.csv',
    '../data/icecube_10year_ps/irfs/IC59_effectiveArea.csv',
    '../data/icecube_10year_ps/irfs/IC79_effectiveArea.csv',
    '../data/icecube_10year_ps/irfs/IC86_I_effectiveArea.csv',
    '../data/icecube_10year_ps/irfs/IC86_II_effectiveArea.csv',
]
eff_dic = {
    0: ice_parser(eff_areas[0]),
    1: ice_parser(eff_areas[1]),
    2: ice_parser(eff_areas[2]),
    3: ice_parser(eff_areas[3]),
    4: ice_parser(eff_areas[4]),
    5: ice_parser(eff_areas[4]),
    6: ice_parser(eff_areas[4]),
    7: ice_parser(eff_areas[4]),
    8: ice_parser(eff_areas[4]),
    9: ice_parser(eff_areas[4]),
}

In [24]:
# MJD, log10(E/GeV), AngErr[deg], RA[deg], Dec[deg], Azimuth[deg], Zenith[deg]
data_sets = [
    '../data/icecube_10year_ps/events/IC40_exp.csv',
    '../data/icecube_10year_ps/events/IC59_exp.csv',
    '../data/icecube_10year_ps/events/IC79_exp.csv',
    '../data/icecube_10year_ps/events/IC86_I_exp.csv',
    '../data/icecube_10year_ps/events/IC86_II_exp.csv',
    '../data/icecube_10year_ps/events/IC86_III_exp.csv',
    '../data/icecube_10year_ps/events/IC86_IV_exp.csv',
    '../data/icecube_10year_ps/events/IC86_V_exp.csv',
    '../data/icecube_10year_ps/events/IC86_VI_exp.csv',
    '../data/icecube_10year_ps/events/IC86_VII_exp.csv',
]
event_dic = {
    0: ice_parser(data_sets[0]),
    1: ice_parser(data_sets[1]),
    2: ice_parser(data_sets[2]),
    3: ice_parser(data_sets[3]),
    4: ice_parser(data_sets[4]),
    5: ice_parser(data_sets[5]),
    6: ice_parser(data_sets[6]),
    7: ice_parser(data_sets[7]),
    8: ice_parser(data_sets[8]),
    9: ice_parser(data_sets[9]),
}

In [25]:
# MJD, log10(E/GeV), AngErr[deg], RA[deg], Dec[deg], Azimuth[deg], Zenith[deg]
uptime_sets = [
    '../data/icecube_10year_ps/uptime/IC40_exp.csv',
    '../data/icecube_10year_ps/uptime/IC59_exp.csv',
    '../data/icecube_10year_ps/uptime/IC79_exp.csv',
    '../data/icecube_10year_ps/uptime/IC86_I_exp.csv',
    '../data/icecube_10year_ps/uptime/IC86_II_exp.csv',
    '../data/icecube_10year_ps/uptime/IC86_III_exp.csv',
    '../data/icecube_10year_ps/uptime/IC86_IV_exp.csv',
    '../data/icecube_10year_ps/uptime/IC86_V_exp.csv',
    '../data/icecube_10year_ps/uptime/IC86_VI_exp.csv',
    '../data/icecube_10year_ps/uptime/IC86_VII_exp.csv',
]
uptime_dic = {
    0: ice_parser(uptime_sets[0]),
    1: ice_parser(uptime_sets[1]),
    2: ice_parser(uptime_sets[2]),
    3: ice_parser(uptime_sets[3]),
    4: ice_parser(uptime_sets[4]),
    5: ice_parser(uptime_sets[5]),
    6: ice_parser(uptime_sets[6]),
    7: ice_parser(uptime_sets[7]),
    8: ice_parser(uptime_sets[8]),
    9: ice_parser(uptime_sets[9]),
}
uptime_tot_dic = {}
for year in range(10):
    uptime_tot_dic[year] = np.sum(np.diff(uptime_dic[year])) * days

In [26]:
# Loading smearing
# log10(E_nu/GeV)_min, log10(E_nu/GeV)_max, Dec_nu_min[deg], Dec_nu_max[deg], log10(E/GeV), PSF_min[deg], PSF_max[deg],
# AngErr_min[deg], AngErr_max[deg], Fractional_Counts
smearing_sets = [
    '../data/icecube_10year_ps/irfs/IC40_smearing.csv',
    '../data/icecube_10year_ps/irfs/IC59_smearing.csv',
    '../data/icecube_10year_ps/irfs/IC79_smearing.csv',
    '../data/icecube_10year_ps/irfs/IC86_I_smearing.csv',
    '../data/icecube_10year_ps/irfs/IC86_II_smearing.csv',
    '../data/icecube_10year_ps/irfs/IC86_II_smearing.csv',
    '../data/icecube_10year_ps/irfs/IC86_II_smearing.csv',
    '../data/icecube_10year_ps/irfs/IC86_II_smearing.csv',
    '../data/icecube_10year_ps/irfs/IC86_II_smearing.csv',
    '../data/icecube_10year_ps/irfs/IC86_II_smearing.csv',
]
smearing_dic = {
    0: ice_parser(smearing_sets[0]),
    1: ice_parser(smearing_sets[1]),
    2: ice_parser(smearing_sets[2]),
    3: ice_parser(smearing_sets[3]),
    4: ice_parser(smearing_sets[4]),
    5: ice_parser(smearing_sets[5]),
    6: ice_parser(smearing_sets[6]),
    7: ice_parser(smearing_sets[7]),
    8: ice_parser(smearing_sets[8]),
    9: ice_parser(smearing_sets[9]),
}

In [27]:
# Loading simulation results
surface_fluxes = pickle.load(open("../data/surf_store_v1.p", "rb"))
# Adding 90 deg
surface_fluxes[90] = surface_fluxes[89]
weights_stored = pickle.load(open("../data/weights_store.p", "rb"))
numu_lib = weights_stored[0]
mu_lib = weights_stored[1]
anti_lib = weights_stored[2]

In [28]:
# Astro
def astro_flux(E):
    # From IceCube data
    # res = 1.44 * (E / 1e5)**(-2.28) * 1e-18
    res = 1.66 * (E / 1e5)**(-2.53) * 1e-18
    return res

In [29]:
def smearing_function(true_e, true_dec, year):
    # Returns the smeared reconstructed values
    e_test = true_e
    angle_test = true_dec
    local_smearing = smearing_dic[year]
    cross_check_smear_egrid = (local_smearing[:, 1] + local_smearing[:, 0])/2.
    idE = np.abs(cross_check_smear_egrid - e_test).argmin()
    all_near_e = (np.where(cross_check_smear_egrid == cross_check_smear_egrid[idE])[0])
    cross_check_smear_theta = (local_smearing[:, 2] + local_smearing[:, 3])/2.
    idtheta = np.abs(cross_check_smear_theta - angle_test).argmin()
    all_near_theta = (np.where(cross_check_smear_theta == cross_check_smear_theta[idtheta])[0])
    elements_of_interest = np.intersect1d(all_near_e, all_near_theta)
    tmp_local_smearing = local_smearing[elements_of_interest]
    smearing_e_grid = np.unique(tmp_local_smearing[:, 4])
    smearing_fraction = []
    for smearing_e_loop in smearing_e_grid:
        idE = np.abs(tmp_local_smearing[:, 4] - smearing_e_loop).argmin()
        all_near_e = (np.where(tmp_local_smearing[:, 4] == tmp_local_smearing[:, 4][idE])[0])
        smearing_fraction.append(np.sum(tmp_local_smearing[all_near_e][:, -1]))
    # Normalizing
    smearing_fraction = np.array(smearing_fraction) / np.trapz(smearing_fraction, x=smearing_e_grid)
    return smearing_e_grid, smearing_fraction

In [30]:
def effective_area_func(surface_fluxes, year):
    # Apply the effective area to the simulation and return unsmeared counts
    cross_check_egrid = (eff_dic[year][:, 1] + eff_dic[year][:, 0])/2.
    cross_check_theta = (eff_dic[year][:, 2] + eff_dic[year][:, 3])/2.
    particle_counts = []
    astro_counts = []
    for theta in list(surface_fluxes.keys()):
        surf_counts = surface_fluxes[theta][-1]  # should only need to multiply with fluxes
        m_egrid = surface_fluxes[theta][0]
        eff_areas = []
        check_angle = theta
        for energy in m_egrid:
            if energy < 1e1:
                eff_areas.append(0.)
            else:
                loge = np.log10(energy)
                idE = np.abs(cross_check_egrid - loge).argmin()
                all_near = (np.where(cross_check_egrid == cross_check_egrid[idE])[0])
                idTheta = np.abs(cross_check_theta[all_near] - check_angle).argmin()
                eff_areas.append(eff_dic[year][all_near, -1][idTheta])
        loc_eff_area = np.array(eff_areas)
        unsmeared_atmos_counts = surf_counts * loc_eff_area * uptime_tot_dic[year] * surface_fluxes[theta][1] * 2. * np.pi
        unsmeared_astro_counts = (
            astro_flux(m_egrid) * loc_eff_area * surface_fluxes[theta][1] * uptime_tot_dic[year] * 2. * np.pi
        )
        particle_counts.append(unsmeared_atmos_counts)
        astro_counts.append(unsmeared_astro_counts)
    return np.array(particle_counts), np.array(astro_counts), m_egrid
def effective_area_func_new(surface_fluxes, new_flux, year):
    # Apply the effective area to the simulation and return unsmeared counts
    cross_check_egrid = (eff_dic[year][:, 1] + eff_dic[year][:, 0])/2.
    cross_check_theta = (eff_dic[year][:, 2] + eff_dic[year][:, 3])/2.
    particle_counts = []
    astro_counts = []
    for theta in list(surface_fluxes.keys()):
        surf_counts = new_flux  # should only need to multiply with fluxes
        m_egrid = surface_fluxes[theta][0]
        eff_areas = []
        check_angle = theta
        for energy in m_egrid:
            if energy < 1e1:
                eff_areas.append(0.)
            else:
                loge = np.log10(energy)
                idE = np.abs(cross_check_egrid - loge).argmin()
                all_near = (np.where(cross_check_egrid == cross_check_egrid[idE])[0])
                idTheta = np.abs(cross_check_theta[all_near] - check_angle).argmin()
                eff_areas.append(eff_dic[year][all_near, -1][idTheta])
        loc_eff_area = np.array(eff_areas)
        unsmeared_atmos_counts = surf_counts * loc_eff_area * surface_fluxes[theta][1] * 2. * np.pi
        unsmeared_astro_counts = (
            astro_flux(m_egrid) * loc_eff_area * surface_fluxes[theta][1] * 2. * np.pi
        )
        particle_counts.append(unsmeared_atmos_counts)
        astro_counts.append(unsmeared_astro_counts)
    return np.array(particle_counts), np.array(astro_counts), m_egrid

In [31]:
def sim_to_dec(surface_fluxes, year):
    # Converts simulation data to detector data
    atmos_counts_unsmeared, astro_counts_unsmeared, m_egrid = effective_area_func(surface_fluxes, year)
    log_egrid = np.log10(m_egrid)
    smeared_atmos = []
    smeared_astro = []
    for id_theta, theta in tqdm(enumerate(list(surface_fluxes.keys()))):
        check_angle = theta
        smeared_atmos_loc = []
        smeared_astro_loc = []
        int_grid = []
        for id_check in range(len(log_egrid)):
            smearing_e, smearing = smearing_function(log_egrid[id_check], check_angle, year)
            if len(smearing) < 3:
                continue
            atmos_spl = UnivariateSpline(smearing_e, smearing * atmos_counts_unsmeared[id_theta][id_check],
                                         k=1, s=0, ext=1)
            astro_spl = UnivariateSpline(smearing_e, smearing * astro_counts_unsmeared[id_theta][id_check],
                                         k=1, s=0, ext=1)
            smeared_atmos_loc.append(atmos_spl(log_egrid))
            smeared_astro_loc.append(astro_spl(log_egrid))
            int_grid.append(log_egrid[id_check])
        smeared_atmos.append(np.trapz(smeared_atmos_loc, x=int_grid, axis=0))
        smeared_astro.append(np.trapz(smeared_astro_loc, x=int_grid, axis=0))
    return np.array(smeared_atmos), np.array(smeared_astro), m_egrid
def sim_to_dec_new_component(surface_fluxes, new_flux, year):
    # Converts simulation data to detector data
    atmos_counts_unsmeared, astro_counts_unsmeared, m_egrid = effective_area_func_new(surface_fluxes, new_flux, year)
    log_egrid = np.log10(m_egrid)
    smeared_atmos = []
    smeared_astro = []
    for id_theta, theta in tqdm(enumerate(list(surface_fluxes.keys()))):
        check_angle = theta
        smeared_atmos_loc = []
        smeared_astro_loc = []
        int_grid = []
        for id_check in range(len(log_egrid)):
            smearing_e, smearing = smearing_function(log_egrid[id_check], check_angle, year)
            if len(smearing) < 3:
                continue
            atmos_spl = UnivariateSpline(smearing_e, smearing * atmos_counts_unsmeared[id_theta][id_check],
                                         k=1, s=0, ext=1)
            astro_spl = UnivariateSpline(smearing_e, smearing * astro_counts_unsmeared[id_theta][id_check],
                                         k=1, s=0, ext=1)
            smeared_atmos_loc.append(atmos_spl(log_egrid))
            smeared_astro_loc.append(astro_spl(log_egrid))
            int_grid.append(log_egrid[id_check])
        smeared_atmos.append(np.trapz(smeared_atmos_loc, x=int_grid, axis=0))
        smeared_astro.append(np.trapz(smeared_astro_loc, x=int_grid, axis=0))
    return np.array(smeared_atmos), np.array(smeared_astro), m_egrid

In [32]:
m_egrid = surface_fluxes[0.][0]

In [33]:
# Totals
# Smeared
injection_energies = m_egrid[45:]
for energy_id, injection_e in tqdm(enumerate(injection_energies)):
    atmos_all = {}
    astro_all = {}
    for year in range(10):
        if year <= 4:
            particle_counts_smeared_unin, astro_counts_smeared_unin, m_egrid_smeared = (
                sim_to_dec_new_component(surface_fluxes, numu_lib[injection_e], year)
            )
            particle_counts_smeared = np.trapz(particle_counts_smeared_unin, x=list(surface_fluxes.keys()), axis=0)
            astro_counts_smeared = np.trapz(astro_counts_smeared_unin, x=list(surface_fluxes.keys()), axis=0)
            atmos_all[year] = particle_counts_smeared * uptime_tot_dic[year]
            astro_all[year] = astro_counts_smeared * uptime_tot_dic[year]
        else:
            atmos_all[year] = atmos_all[4] / uptime_tot_dic[4] * uptime_tot_dic[year]
            astro_all[year] = astro_all[4] / uptime_tot_dic[4] * uptime_tot_dic[year]
    total_atmos = atmos_all[0]
    for year in range(10):
        if year == 0:
            continue
        total_atmos += atmos_all[year]
    pickle.dump([total_atmos, injection_energies], open("../data/injection_lib/simulated_data_inj_store_v2_%d.p" % energy_id, "wb"))

0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:03,  3.58s/it]
2it [00:07,  3.75s/it]
3it [00:11,  3.69s/it]
4it [00:14,  3.64s/it]
5it [00:18,  3.60s/it]
6it [00:24,  4.44s/it]
7it [00:29,  4.83s/it]
8it [00:37,  5.80s/it]
9it [00:44,  6.05s/it]
10it [00:49,  5.64s/it]
11it [00:52,  4.90s/it]
12it [00:55,  4.41s/it]
13it [01:03,  5.34s/it]
14it [01:10,  6.04s/it]
15it [01:16,  6.04s/it]
16it [01:20,  5.36s/it]
17it [01:28,  6.15s/it]
18it [01:35,  6.38s/it]
19it [01:39,  5.74s/it]
20it [01:43,  5.20s/it]
21it [01:48,  5.22s/it]
22it [01:52,  4.77s/it]
23it [01:56,  4.45s/it]
24it [02:02,  4.90s/it]
25it [02:06,  4.71s/it]
26it [02:11,  4.68s/it]
27it [02:16,  4.79s/it]
28it [02:24,  5.16s/it]
0it [02:24, ?it/s]


KeyboardInterrupt: 